<a href="https://colab.research.google.com/github/zolduoarrati/Python/blob/master/PEGASUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.5 MB/s 


In [2]:
pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
     |████████████████████████████████| 636 kB 42.6 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [4]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [13]:
# Paragraph of text
context = "Similarly, heterogeneity has been described as an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations or environments. Campbell et al. (2011) show that introducing variation in agent behaviour to achieve dynamic task allocation increases the system’s ability to adapt to varying stimuli. Ferrari (2010, pp. 39–59) recommend heterogeneity in wireless sensor networks as a mechanism for generating near-optimal configurations. Lewis et al. (2013) demonstrate that heterogeneity in distributed smart camera networks can lead to new highly efficient configurations."
print(context)

Similarly, heterogeneity has been described as an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations or environments. Campbell et al. (2011) show that introducing variation in agent behaviour to achieve dynamic task allocation increases the system’s ability to adapt to varying stimuli. Ferrari (2010, pp. 39–59) recommend heterogeneity in wireless sensor networks as a mechanism for generating near-optimal configurations. Lewis et al. (2013) demonstrate that heterogeneity in distributed smart camera networks can lead to new highly efficient configurations.


In [14]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['Similarly, heterogeneity has been described as an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations or environments.',
 'Campbell et al. (2011) show that introducing variation in agent behaviour to achieve dynamic task allocation increases the system’s ability to adapt to varying stimuli.',
 'Ferrari (2010, pp. 39–59) recommend heterogeneity in wireless sensor networks as a mechanism for generating near-optimal configurations.',
 'Lewis et al. (2013) demonstrate that heterogeneity in distributed smart camera networks can lead to new highly efficient configurations.']

In [15]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [16]:
# This is the paraphrased text
paraphrase

[['Heterogeneity is an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations.'],
 ['The ability to adapt to different stimuli is increased by introducing variation in agent behavior.'],
 ['Heterogeneity in wireless sensor networks is a mechanism for generating near-optimal configurations.'],
 ['Heterogeneity in smart camera networks can lead to new highly efficient configurations.']]

In [17]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['Heterogeneity is an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations.',
 'The ability to adapt to different stimuli is increased by introducing variation in agent behavior.',
 'Heterogeneity in wireless sensor networks is a mechanism for generating near-optimal configurations.',
 'Heterogeneity in smart camera networks can lead to new highly efficient configurations.']

In [18]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'Heterogeneity is an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations. The ability to adapt to different stimuli is increased by introducing variation in agent behavior. Heterogeneity in wireless sensor networks is a mechanism for generating near-optimal configurations. Heterogeneity in smart camera networks can lead to new highly efficient configurations.'

In [19]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)
print(context)
print(paraphrased_text)

Similarly, heterogeneity has been described as an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations or environments. Campbell et al. (2011) show that introducing variation in agent behaviour to achieve dynamic task allocation increases the system’s ability to adapt to varying stimuli. Ferrari (2010, pp. 39–59) recommend heterogeneity in wireless sensor networks as a mechanism for generating near-optimal configurations. Lewis et al. (2013) demonstrate that heterogeneity in distributed smart camera networks can lead to new highly efficient configurations.
Heterogeneity is an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations. The ability to adapt to different stimuli is increased by introducing variation in agent behavior. Heterogeneity in wireless sensor networks is a mechanism for generating near-optimal configurations. Heterogeneity in smart camera networks ca

In [22]:
# Paragraph of text
context = "Heterogeneity is an important characteristic of multi-agent and self-organising systems, allowing them to adapt to unknown situations. The ability to adapt to different stimuli is increased by introducing variation in agent behavior. Heterogeneity in wireless sensor networks is a mechanism for generating near-optimal configurations. Heterogeneity in smart camera networks can lead to new highly efficient configurations."
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences
splitter = SentenceSplitter(language='en')
sentence_list = splitter.split(context)
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []
for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [23]:
paraphrase2 = [' '.join(x) for x in paraphrase]
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
print(paraphrased_text)

Heterogeneity is an important characteristic of multi-agent and self-organising systems. Variation in agent behavior increases the ability to adapt to different stimuli. Heterogeneity is a mechanism for generating configurations. Heterogeneity in smart camera networks can lead to new configurations.
